In [1]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

ModuleNotFoundError: No module named 'tensorflow.python.tools'; 'tensorflow.python' is not a package

In [ ]:
data = 'horses_or_humans'
builder = tfds.builder(data)
info = builder.info

In [ ]:
print(info)

In [ ]:
#prep
training_data = tfds.load(name=data, split='train')
test_data = tfds.load(name=data, split='test')


In [ ]:
#thinking

some_picture = tfds.show_examples(info, training_data)


In [ ]:
#more prep pgn to numpy array
train_images = np.array([example['image'].numpy()[:,:,0] for example in training_data])
train_labels = np.array([example['label'].numpy() for example in training_data])

test_images = np.array([example['image'].numpy()[:,:,0] for example in test_data])
test_labels = np.array([example['label'].numpy() for example in test_data])


In [ ]:
print(train_images.shape)
print(test_images.shape)

In [ ]:
train_images = train_images.reshape(1027, 300, 300, 1)
test_images = test_images.reshape(256, 300, 300, 1)

train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

train_images /= 255
test_images /= 255

In [ ]:
#create model
model = keras.Sequential([
  keras.layers.Flatten(),
  keras.layers.Dense(512, activation='relu'),
  keras.layers.Dense(256, activation='relu'),
  keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10, batch_size=32)


In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
#convolutional

model = keras.Sequential([
  keras.layers.Conv2D(80, 2, activation='relu', input_shape=(300,300,1)),
  keras.layers.Conv2D(40, 2, activation='relu'),
  keras.layers.Flatten(),
  keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10, batch_size=32)



In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
#2.nd conv
model = keras.Sequential([
  keras.layers.AveragePooling2D(6,3, input_shape=(300,300,1)),
  keras.layers.Conv2D(100, 2, activation='relu'),
  keras.layers.Conv2D(50, 2, activation='relu'),
  keras.layers.MaxPool2D(2,2),
  keras.layers.Dropout(0.5),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10, batch_size=32)



In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
pip install -U keras-tuner

In [ ]:
#hyperparameter
from kerastuner.tuners import RandomSearch

def x_model(hp):
  model = keras.Sequential()

  model.add(keras.layers.AveragePooling2D(6,3,input_shape=(300,300,1)))

  for i in range(hp.Int("Conv Layers", min_value=0, max_value=3)):
    model.add(keras.layers.Conv2D(hp.Choice(f"layer_{i}_filters", [16,32,64]), 3, activation='relu')) 
  model.add(keras.layers.MaxPool2D(2,2))
  model.add(keras.layers.Dropout(0.40))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(hp.Choice("Dense layer", [64, 128, 256, 512, 1024]), activation='relu'))
  model.add(keras.layers.Dense(3, activation='softmax'))
  model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
  return model

tuner = RandomSearch(
    x_model,
    objective='val_accuracy',
    max_trials=32,
)

tuner.search(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10, batch_size=32)

In [ ]:
best_model = tuner.get_best_models()[0]

In [ ]:
best_model.evaluate(test_images, test_labels)

In [ ]:
best_model.summary()

In [ ]:
tuner.results_summary()

In [ ]:
best_model.save('./my_model')

In [ ]:
loaded_model = keras.models.load_model('./my_model')

In [ ]:
loaded_model.evaluate(test_images, test_labels)

In [ ]:
#plot img from numpy array
rgb_images = np.array([example['image'].numpy() for example in training_data.take(1)])
rgb_image = rgb_images[0]

image = train_images[0].reshape(300,300)

plt.imshow(rgb_image)

rgb_image.shape

In [ ]:
result = best_model.predict(np.array([train_images[0]]))
print(result)

predicted_value = np.argmax(result)
print(predicted_value)

In [ ]:
#vonvert png/jpg to numpy format
import imageio

im = imageio.imread('.jpg')

print(type(im))

im_np = np.asarray(im)

print(im_np.shape)